# Uso definitivo del modelo

$$
\begin{align}
\frac{dS}{dt}=\xi_{1}M(t)+\xi_{2}E(t)-\xi_{3}S(t) \\
\frac{dE}{dt}=\xi_{4}M(t)+\xi_{5}S(t)-\xi_{6}E(t)
\end{align}
$$

In [1]:
# cargar bases de datos de ambos paises
import pandas as pd

df = pd.read_csv('../../data/processed/salud/unified.csv')
print(df.head())


   Unnamed: 0   Año       USD mex        USD ch   dG/dt (mex)    dG/dt (ch)
0           0  2004  3.437250e+10  6.493000e+09  0.000000e+00  0.000000e+00
1           1  2005  3.443720e+10  6.944000e+09  6.470000e+07  4.510000e+08
2           2  2006  3.845950e+10  8.485000e+09  4.022300e+09  1.541000e+09
3           3  2007  4.166450e+10  9.633000e+09  3.205000e+09  1.148000e+09
4           4  2008  4.561570e+10  1.063700e+10  3.951200e+09  1.004000e+09


In [4]:
# leer las bases de datos para los egresos 
df_mex=pd.read_csv('../../data/processed/salud/egresos_méxico.csv')
df_ch=pd.read_csv('../../data/processed/salud/egresos_chile.csv')

df['E mex']=df_mex['Egresos']
df['E ch']=df_ch['Egresos']
df = df.drop('Unnamed: 0',axis=1)

print(df.head())

    Año       USD mex        USD ch   dG/dt (mex)    dG/dt (ch)    E mex  \
0  2004  3.437250e+10  6.493000e+09  0.000000e+00  0.000000e+00  4416313   
1  2005  3.443720e+10  6.944000e+09  6.470000e+07  4.510000e+08  4539477   
2  2006  3.845950e+10  8.485000e+09  4.022300e+09  1.541000e+09  4658305   
3  2007  4.166450e+10  9.633000e+09  3.205000e+09  1.148000e+09  4922227   
4  2008  4.561570e+10  1.063700e+10  3.951200e+09  1.004000e+09  5123675   

     E ch  
0  248235  
1  248169  
2  278142  
3  299953  
4  315315  


In [9]:
# cambios

df['dE/dt mex']=df['E mex'].diff().fillna(0)
df['dE/dt ch']=df['E ch'].diff().fillna(0)

df = df.iloc[:17]
print(df.head())
df.to_csv('../../data/processed/salud/unified_mod.csv')

    Año       USD mex        USD ch   dG/dt (mex)    dG/dt (ch)    E mex  \
0  2004  3.437250e+10  6.493000e+09  0.000000e+00  0.000000e+00  4416313   
1  2005  3.443720e+10  6.944000e+09  6.470000e+07  4.510000e+08  4539477   
2  2006  3.845950e+10  8.485000e+09  4.022300e+09  1.541000e+09  4658305   
3  2007  4.166450e+10  9.633000e+09  3.205000e+09  1.148000e+09  4922227   
4  2008  4.561570e+10  1.063700e+10  3.951200e+09  1.004000e+09  5123675   

     E ch  dE/dt mex  dE/dt ch  
0  248235        0.0       0.0  
1  248169   123164.0     -66.0  
2  278142   118828.0   29973.0  
3  299953   263922.0   21811.0  
4  315315   201448.0   15362.0  


In [20]:
# modelo mexico
import statsmodels.api as sm

# cargar datos de poblacion mayores
pobmex=pd.read_csv('../../data/processed/poblaciones/mexico.csv').iloc[2:19].reset_index()['Poblacion adultos mayores (65+)']

# salud

X_s=pd.DataFrame({
    'poblacion':pobmex.values,
    'egresos':df['E mex'].values,
    'salud': df['USD mex'].values,
    
})
Y_s=df['dG/dt (mex)']
X_s=sm.add_constant(X_s,prepend=False)

modelo_s=sm.OLS(Y_s,X_s).fit()
print(modelo_s.summary())

# egresos

X_e=pd.DataFrame({
    'poblacion' : pobmex.values,
    'salud': df['USD mex'].values,
    'egresos' : df['E mex'].values,
})
Y_e=df['dE/dt mex']
X_e=sm.add_constant(X_e,prepend=False)

modelo_e=sm.OLS(Y_e,X_e).fit()
print(modelo_e.summary())



                            OLS Regression Results                            
Dep. Variable:            dG/dt (mex)   R-squared:                       0.700
Model:                            OLS   Adj. R-squared:                  0.631
Method:                 Least Squares   F-statistic:                     10.13
Date:                Wed, 26 Nov 2025   Prob (F-statistic):            0.00104
Time:                        13:59:07   Log-Likelihood:                -384.50
No. Observations:                  17   AIC:                             777.0
Df Residuals:                      13   BIC:                             780.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
poblacion  -4879.1222   1270.124     -3.841      0.0

In [ ]:
# Graficacion


def sistema_edos(t, y):
    # y = [J, A, M, S, E]
    jt, at, mt, st, et = y

    # COEFICIENTES modelo poblacion
    c1 = -0.0085  # tasa de natalidad
    c2 = -0.0207  # tasa de mortalidad jóvenes
    #const1= -2.161e+06
    tj=0.0359
    c3 = -0.0036  # tasa de mortalidad adultos (positiva)
    #const2=-3.146e+07
    ta=-0.0002
    c4 =  -0.0344 # tasa de mortalidad mayores
    #const3=-1.404e+06

    xi1=0
    xi2=0
    xi3=0
    xi4=0
    xi5=0
    xi6=0
    const4=0
    const5=0
    

    # ECUACIONES Poblacion
    dyJ = c1*at-c2*jt#+const1
    dyA = tj*jt-c3*at#+const2
    dyM = ta*at-c4*mt#+const3

    # Ecuaciones salud

    dyS=xi1*mt+xi2*et-xi3*st+const4
    dyE=xi4*mt+xi5*st-xi6*et+const5

    return [dyJ, dyA, dyE, dyS, dyE]



In [ ]:
import numpy as np
def RK4(func, y0, t0, tf, h):
    t_values = np.arange(t0, tf + h, h)
    n = len(t_values)
    y_values = np.zeros((n, len(y0)), dtype=float)
    y_values[0] = y0

    for i in range(1, n):
        k1 = np.array(func(t_values[i-1], y_values[i-1]))
        k2 = np.array(func(t_values[i-1] + h/2, y_values[i-1] + h*k1/2))
        k3 = np.array(func(t_values[i-1] + h/2, y_values[i-1] + h*k2/2))
        k4 = np.array(func(t_values[i-1] + h,   y_values[i-1] + h*k3))
        y_values[i] = y_values[i-1] + h*(k1 + 2*k2 + 2*k3 + k4)/6

    return t_values, y_values

# y0 = [J, A, E, S, E] en 2005
y0 = [27858970,65630336,5953231, 34372500000.0, 4416313]
t0, tf, h = 0, 15, 0.01

t, y = RK4(sistema_edos, y0, t0, tf, h)